In [ ]:
# helper method for col cleaning
years = np.linspace(2015, 2020, 6, dtype=int).tolist()
table4 = []
for y in years:
    n = y-2015
    table4_path='./Data_%s/CSV_files_%s/Table_4_Offenses_Offense_Type_by_Bias_Motivation_%s.csv'%(y,y,y)
    table4.append(pd.read_csv(table4_path, thousands=','))
table4[5]

base_col = table4[5].columns.tolist()
base_row = table4[5]['Bias_motivation'].tolist()
print(base_row)

i = 2015
for t in table4:
    for r in t['Bias_motivation']:
        if r not in base_row: print ('row: '+r,i)
    i += 1

j = 2015
for t in table4:
    for c in t.columns:
        if c not in base_col: print ('col: '+c,j)
    j += 1

In [ ]:
# regression
def reg(df, cat):
    X = df[['Year', cat]]
    y = df[['Total_offenses']]
    m1 = LinearRegression().fit(X,y)
    yhat = m1.predict(X)
    print('intercept: '+str(m1.intercept_))
    print('coefficient: '+str(m1.coef_))
    df[['Total_offenses']] = yhat
    ax = sns.scatterplot(data=df, x='Year', y='Total_offenses', hue=cat)
    sns.move_legend(ax, 'upper left', bbox_to_anchor=(1,1))
    plt.show()

def reg_plot(cats):
    cat_list = cats.to_list()
    cat_list = list(dict.fromkeys(cat_list))
    for cat in cat_list:
        cat_name = 'is_' + str(cat)
        reg(table_1, cat_name)

In [ ]:
# pie charts
def plot_pie(df):
    df = df.drop(['Bias_motivation', 'Total_offenses', 'Year', 'Category'],axis=1)

    data = df.iloc[2,:].to_numpy().tolist()
    labels = df.columns.to_list()
    colors = sns.color_palette('pastel')[0:15]
    fig, ax = plt.subplots()

    # plt.pie(data, labels = labels, colors = colors, autopct='%.1f%%')
    l = ax.pie(data, autopct='%.1f%%', startangle=-90)

    for label, t in zip(labels, l[1]):
        x, y = t.get_position()
        angle = int(math.degrees(math.atan2(y, x)))
        ha = "left"

        if x<0:
            angle -= 180
            ha = "right"

        plt.annotate(label, xy=(x,y), rotation=angle, ha=ha, va="center", rotation_mode="anchor", size=8)

    plt.show()
    
table_all.groupby('Year').apply(plot_pie)

In [ ]:
# wrong initial appraoch
%sql offense_type << SELECT * FROM table1_19 FULL OUTER JOIN table4_19 ON table1_19.Offenses = table4_19.Total_Offenses

offense_type = offense_type.loc[:,~offense_type.columns.duplicated()]
offense_type = offense_type.drop(['Total_offenses'],axis=1)
offense_type.head()

t4_19 = table4_19.drop(['Bias_motivation', 'Total_offenses'],axis=1)
sns.barplot(data=t4_19.iloc[1:2], orient = 'h')
plt.show()

# Simplify Table 4
%sql table4_16_type << SELECT Bias_motivation, Total_offenses AS '2016' FROM table4_16 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')
%sql table4_17_type << SELECT Bias_motivation, Total_offenses AS '2017' FROM table4_17 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')
%sql table4_18_type << SELECT Bias_motivation, Total_offenses AS '2018' FROM table4_18 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')
%sql table4_19_type << SELECT Bias_motivation, Total_offenses AS '2019' FROM table4_19 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')

# Combine Table 4 from 2016-2019
combined_table4 = pd.concat([table4_16_type, table4_17_type, table4_18_type, table4_19_type], axis=1)
combined_table4 = combined_table4.loc[:,~combined_table4.columns.duplicated()].copy()
combined_table4

tidy_table4['Year'] = tidy_table4['Year'].astype(int)
sns.lmplot(x='Year', y='Total_offenses', data=tidy_table4, hue='Bias_motivation')
plt.show()

def plot_by_bm(df):
    df['Year'] = df['Year'].astype(float)
    sns.lmplot(x='Year', y='Total_offenses', data=df, hue='Bias_motivation')
    plt.show()

tidy_table4.groupby(['Bias_motivation']).apply(plot_by_bm)

In [ ]:
# Plot output~one input groupy by State
def plot_single(df, input):
    sns.lmplot(x=input, y='Incidents',  col='State', data=df, hue='State')
    plt.show()

# Plot each input
for i in input:
    plot_single(table, i)

In [ ]:
# Print reg model variables, coefficient, and intercept
def run_regression(input, train, test, output):
    X_train = train[input]
    X_test = test[input]
    y_train = train[output]
    y_test = test[output]
    reg = LinearRegression().fit(X_train, y_train)

    pred_train = reg.predict(X_train)
    pred_test = reg.predict(X_test)

    print('variables: '+str(input))
    print('coefficient: '+str(reg.coef_))
    print('intercept: '+str(reg.intercept_))

    train_mse = (np.square(pred_train - y_train)).mean()
    test_mse = (np.square(pred_test - y_test)).mean()
    print('train_rmse: '+ str(math.sqrt(train_mse)))
    print('test_rmse: '+ str(math.sqrt(test_mse)))

    train_mae = (np.abs(pred_train - y_train)).mean()
    test_mae = (np.abs(pred_test - y_test)).mean()
    print('train_mae: '+ str(train_mae))
    print('test_mae: '+ str(test_mae))
    print(' ')

In [ ]:
# [UNFINISHED] visualization
states = geopandas.read_file('geopandas-tutorial/data/usa-states-census-2014.shp')
states.plot()
plt.show()

In [ ]:
# KFold to verify variable selection
def kfold(input):
    kf = KFold(n_splits=10, shuffle=True)
    X = train[input]
    reg = LinearRegression()
    scores = cross_val_score(reg, X, inc, scoring='neg_mean_absolute_error', cv=kf, n_jobs=-1)
    acc = cross_val_score(reg, X, inc, cv=kf)
    m = sqrt(mean(absolute(scores)))
    print('KFold'+str(input)+'RMSE: '+str(m))
    # print('accuracy: '+str(acc.mean())+' std: '+str(acc.std()) )

In [ ]:
list = ['Alabama'
,'Alaska'
,'Arizona'
,'Arkansas'
,'California'
,'Colorado'
,'Connecticut'
,'Delaware'
,'Florida'
,'Georgia'
,'Hawaii'
,'Idaho'
,'Illinois'
,'Indiana'
,'Iowa'
,'Kansas'
,'Kentucky'
,'Louisiana'
,'Maine'
,'Maryland'
,'Massachusetts'
,'Michigan'
,'Minnesota'
,'Mississippi'
,'Missouri'
,'Montana'
,'Nebraska'
,'Nevada'
,'New Hampshire'
,'New Jersey'
,'New Mexico'
,'New York'
,'North Carolina'
,'North Dakota'
,'Ohio'
,'Oklahoma'
,'Oregon'
,'Pennsylvania'
,'Rhode Island'
,'South Carolina'
,'South Dakota'
,'Tennessee'
,'Texas'
,'Utah'
,'Vermont'
,'Virginia'
,'Washington'
,'West Virginia'
,'Wisconsin'
,'Wyoming']

for l in list:
    d = df.loc[df['State'] == l]
    print(l+str(d.shape[0]))

df.loc[df['State'] == 'Mississippi']

In [ ]:
var = ['Temperature', 'Log_GDP', 'Marriage_Rate', 'Education_Rate']
inc = train['Incidents']

# Anova F test for single variable selection
def anova(var):
    f = 'Incidents ~ ' + var
    d = smf.ols(formula=f, data=train)
    model = d.fit()
    a = sm.stats.anova_lm(model)
    print(a)
    
for v in var: anova(v)

**Analysis**
We first ran ANOVA F-tests on each of the four single-predictor models (X=’Log_GDP’ / ‘Temperature’ / ‘Marriage Rate’ / ‘Education Rate’, y=’Incidents’), since F-stats shows their improvement in significance compared to the intercept-only (null) model. The results show that ’Log_GDP’, ‘Temperature’, and ‘Marriage Rate’ have a p-value less than 0.05, which is significant; among these predictors, log GDP has the smallest p-value, which is 7.04-68. Therefore, we choose log GDP as the input of our one-variable model.

**Evaluation of significance**
For our model with log GDP as predictor, we ran t-test in OLS regression for verification. We got p-values less than 0.05 for both training and testing sets, which are significant results. Therefore, we confirm that our choice of **log GDP is significant** to predict hate crime incidents, and **log GDP is our best single predictor**.

**1.2 Determine the combination of more than one predictors that best predicts the hate crime incidents.** 

**Methods** We conducted step-wise model selection by using Log Likelihood Ratio test, and then use OLS regression to check the model.

- On top of the best one-predictor model, we ran log likelihood ratio tests to determine if we can improve it by adding one more predictor. We iterated this process from raising one predictor to two, two predictors to three, and finally three predictors to four, until we found the optimal combination of predictors for the model. 
 
- In our case, the log likelihood ratio tests if the performance of our previously selected model changes significantly after adding one parameter. Specifically, we compare the best model so far (with variables selected from previous tests) and a new model in which we added a new variable. If the p-value of log likelihood ratio is less than 0.5, which is a significant difference, we argue that adding this variable has the potential to improve the model’s goodness of fit. After selecting a new model, we iterated this process by introducing another predictor, and deciding whether to include it until we found the best combination of inputs.

- After running the Log Likelihood Ratio test, we ran OLS regression on the model with the selected predictors to verify that each input in our newly-selected model has a significant effect on the output (p < 0.05). If all input has p-values less than .05, we confirm that the newly-selected model is the best model so far.
the additional one variable we selected has the most significance in improving model performance.

**Analysis** We tested three possible two-predictors models by using the log likelihood ratio test. The results show that adding a second predictor to our best one-predictor model (X = log GDP) does improve the model’s goodness of fit. The p-values for all three possible two-predictors models are less than 0.05. However, the model with **log GDP and temperature** is the two-predictors model with the lowest p-value, which is 5.94e-10. Thus we selected his two-predictors model as our new best model. 

**Evaluation of significance** We also ran the OLS regression that fits a regression model on log GDP and temperature for both testing and training data. The results show that the p-values are less than 0.05,  which means the predictors in the current two-predictors model are statistically significant. This confirms that the model with inputs **(X = log GDP + Temperature)** is better than the previous model and we should include both inputs. 

**Interpretation**
- Summary:
  - When GDP increases 1 million dollars, hate crime increases 66 incidents.
  - When Year Average Temperature increases 1 Fahrenheit, hate crime decreases 8 incidents.
  - When Education Rate increases 1 %, hate crime decreases 7 incidents.
  - When Marriage increases 1 unit, hate crime decreases 4 incidents.
  
- Prediction: 
  - Vermont at 2020 when 
  - GDP = 29064.6 million dollars, 
  - Year Average Temperature = 44.8 Fahrenheit, 
  - Marriage Rate = 6.0%, 
  - Education Rate = 42.0%, 
  - hate crime incidents is predicted to be 51.16, while the actual incidents number is 60.
- Oddities: 
  - Hate crime incidents should be integers in real world, but our predictions are floats.
  - Hate crime incidents should be all positive in real world, but we sometimes get negative prediction results.

In [ ]:
tg = sm.OLS(inc, train[['Temperature', 'Log_GDP']]).fit()
llr(tg, ['Temperature', 'Log_GDP', 'Marriage_Rate'])
llr(tg, ['Temperature', 'Log_GDP', 'Education_Rate'])
llr(tg, ['Temperature', 'Log_GDP', 'Marriage_Rate', 'Education_Rate'])

summary(train, ['Temperature', 'Log_GDP'])
summary(train, ['Temperature', 'Log_GDP', 'Marriage_Rate'])
summary(train, ['Temperature', 'Log_GDP', 'Education_Rate'])
summary(train, ['Temperature', 'Log_GDP', 'Marriage_Rate', 'Education_Rate'])

repeat(['Temperature', 'Log_GDP'], 200)
repeat(['Temperature', 'Log_GDP', 'Is_Covid'], 200)

In [ ]:
# Plot the correlation between each input (predictor) and the output (incidents)
var = train.columns[3:8]
for v in var:
    sns.lmplot(data=train, x=v, y='Incidents')
    plt.show()

**Linearity check** There's a linear relationship between predictors and response variables.
  
We used scatter plots to check the linear relationship between the predictors and response variables. After we created the scatter plots, we recognized that the plots for population and GDP are non-linear. This means that these two predictors failed to meet the linearity assumption. Therefore, we decided to transform these predictors by taking the logarithm of these predictors before we fit them into our multilinear regression model. In future analysis, the population and GDP we referred to are actually log population and log GDP.

In [ ]:
varl = []
import itertools
for l in range(len(var) + 1):
    for sub in itertools.combinations(var, l):
        varl.append(sub)

In [ ]:
# ANOVA F test for significance evaluation
def anova(f1, f2):
    d1 = smf.ols(formula=f1, data=train)
    model1 = d1.fit()
    d2 = smf.ols(formula=f2, data=train)
    model2 = d2.fit()
    a = sm.stats.anova_lm(model1, model2)
    print(a)

# Moreover, F test (p-value) for evaluation of significance 
anova('Incidents ~ Temperature + Log_GDP + Education_Rate', 'Incidents ~ Temperature + Log_GDP + Education_Rate + Marriage_Rate')

In [ ]:
# Double check: rmse doesn't change significantly after adding Is_Covid
# The performance is even worse because rmse is larger when Is_Covid is included in the predictors.
repeat(['Temperature', 'Log_GDP', 'Education_Rate'], 100)
repeat(['Temperature', 'Log_GDP', 'Education_Rate', 'Is_Covid'], 100)